In [1]:
#biblioteka odpowiedzialna za zapis w hdf5

In [2]:
!pip install pyyaml h5py


[notice] A new release of pip is available: 23.1.2 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


In [3]:
!pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 34.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [4]:
import os
import tensorflow as tf
from tensorflow import keras

In [5]:
(train_images,train_labels),(test_images,test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1,28*28)/255.0
test_images = test_images[:1000].reshape(-1,28*28)/255.0

11490434/11490434 [==============================] - 1s 0us/step


In [6]:
def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512,activation='relu',input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10)
    ])
    model.compile(optimizer='adam',
                  loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    return model

In [7]:
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [8]:
#zapis wag -> plik ckpt
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

model.fit(train_images,
          train_labels,
          epochs=10,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback])

Epoch 1/10
30/32 [===========================>..] - ETA: 0s - loss: 1.1715 - accuracy: 0.6677
Epoch 1: saving model to training_1/cp.ckpt
32/32 [==============================] - 2s 32ms/step - loss: 1.1472 - accuracy: 0.6760 - val_loss: 0.7315 - val_accuracy: 0.7720
Epoch 2/10
16/32 [==============>...............] - ETA: 0s - loss: 0.4858 - accuracy: 0.8535
Epoch 2: saving model to training_1/cp.ckpt
32/32 [==============================] - 1s 17ms/step - loss: 0.4335 - accuracy: 0.8700 - val_loss: 0.5737 - val_accuracy: 0.8180
Epoch 3/10
29/32 [==========================>...] - ETA: 0s - loss: 0.2706 - accuracy: 0.9267
Epoch 3: saving model to training_1/cp.ckpt
32/32 [==============================] - 1s 18ms/step - loss: 0.2762 - accuracy: 0.9250 - val_loss: 0.4670 - val_accuracy: 0.8510
Epoch 4/10
17/32 [==============>...............] - ETA: 0s - loss: 0.1946 - accuracy: 0.9596
Epoch 4: saving model to training_1/cp.ckpt
32/32 [==============================] - 1s 18ms/step - lo

In [9]:
os.listdir(checkpoint_dir)

['checkpoint', 'cp.ckpt.data-00000-of-00001', 'cp.ckpt.index']

In [10]:
model = create_model()
loss,acc = model.evaluate(test_images,test_labels,verbose=2)
print(f'niewytrenowany model: acc -> {acc}, loss -> {loss}')

32/32 - 0s - loss: 2.3543 - accuracy: 0.1010 - 280ms/epoch - 9ms/step
niewytrenowany model: acc -> 0.10100000351667404, loss -> 2.354332447052002


In [11]:
model.load_weights(checkpoint_path)
loss,acc = model.evaluate(test_images,test_labels,verbose=2)
print(f'model z dodanymi wagami: acc -> {acc}, loss -> {loss}')

32/32 - 0s - loss: 0.4106 - accuracy: 0.8690 - 74ms/epoch - 2ms/step
model z dodanymi wagami: acc -> 0.8690000176429749, loss -> 0.4106365144252777


In [12]:
#zapis wielu wag, 50 epok-> zapis co 5 epokę
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
batch_size = 32

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1,
                                                 save_freq=5*batch_size)

model = create_model()

model.save_weights(checkpoint_path.format(epoch=0))

model.fit(train_images,
          train_labels,
          epochs=50,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback],
          batch_size=batch_size,
          verbose=0)


Epoch 5: saving model to training_2/cp-0005.ckpt

Epoch 10: saving model to training_2/cp-0010.ckpt

Epoch 15: saving model to training_2/cp-0015.ckpt

Epoch 20: saving model to training_2/cp-0020.ckpt

Epoch 25: saving model to training_2/cp-0025.ckpt

Epoch 30: saving model to training_2/cp-0030.ckpt

Epoch 35: saving model to training_2/cp-0035.ckpt

Epoch 40: saving model to training_2/cp-0040.ckpt

Epoch 45: saving model to training_2/cp-0045.ckpt

Epoch 50: saving model to training_2/cp-0050.ckpt


In [13]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [14]:
model = create_model()
model.load_weights(latest)

loss,acc = model.evaluate(test_images,test_labels,verbose=2)
print(f'model z ostatniego ckpt: acc -> {acc}, loss -> {loss}')

32/32 - 0s - loss: 0.4830 - accuracy: 0.8720 - 268ms/epoch - 8ms/step
model z ostatniego ckpt: acc -> 0.871999979019165, loss -> 0.4830023944377899


In [15]:
#ręcznie zapisywane wagi
model.save_weights('./checpoints/my_checkpoint')

model = create_model()
model.load_weights('./checpoints/my_checkpoint')

loss,acc = model.evaluate(test_images,test_labels,verbose=2)
print(f'model z własnego cp: acc -> {acc}, loss -> {loss}')

32/32 - 0s - loss: 0.4830 - accuracy: 0.8720 - 272ms/epoch - 8ms/step
model z własnego cp: acc -> 0.871999979019165, loss -> 0.4830023944377899


In [16]:
#zapis całego modelu
model = create_model()
model.fit(train_images,train_labels,epochs=5)

!mkdir -p saved_model
model.save('saved_model/mymodel')

Epoch 1/5
32/32 [==============================] - 1s 4ms/step - loss: 1.1844 - accuracy: 0.6540
Epoch 2/5
32/32 [==============================] - 0s 4ms/step - loss: 0.4216 - accuracy: 0.8790
Epoch 3/5
32/32 [==============================] - 0s 3ms/step - loss: 0.2920 - accuracy: 0.9330
Epoch 4/5
32/32 [==============================] - 0s 3ms/step - loss: 0.2117 - accuracy: 0.9530
Epoch 5/5
32/32 [==============================] - 0s 4ms/step - loss: 0.1535 - accuracy: 0.9680
INFO:tensorflow:Assets written to: saved_model/mymodel/assets


In [17]:
new_model = tf.keras.models.load_model('saved_model/mymodel')
new_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 512)               401920    
                                                                 
 dropout_5 (Dropout)         (None, 512)               0         
                                                                 
 dense_11 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [18]:
loss,acc = model.evaluate(test_images,test_labels,verbose=2)
print(f'model odtworzony z zapisanego modelu: acc -> {acc}, loss -> {loss}')

32/32 - 0s - loss: 0.4027 - accuracy: 0.8720 - 193ms/epoch - 6ms/step
model odtworzony z zapisanego modelu: acc -> 0.871999979019165, loss -> 0.4026578664779663


In [19]:
model = create_model()
model.fit(train_images,train_labels,epochs=5)
model.save('mm.h5')

Epoch 1/5
32/32 [==============================] - 1s 4ms/step - loss: 1.1924 - accuracy: 0.6490
Epoch 2/5
32/32 [==============================] - 0s 3ms/step - loss: 0.4511 - accuracy: 0.8720
Epoch 3/5
32/32 [==============================] - 0s 3ms/step - loss: 0.2922 - accuracy: 0.9170
Epoch 4/5
32/32 [==============================] - 0s 3ms/step - loss: 0.2080 - accuracy: 0.9510
Epoch 5/5
32/32 [==============================] - 0s 3ms/step - loss: 0.1617 - accuracy: 0.9630


In [20]:
next_model = tf.keras.models.load_model('mm.h5')
next_model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 512)               401920    
                                                                 
 dropout_6 (Dropout)         (None, 512)               0         
                                                                 
 dense_13 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________
